In [ ]:
import ast

import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn
import sklearn
import umap
import umap.plot


In [67]:
df = pd.read_csv("data/complete_features.csv", index_col=0)
df = df.set_index("track_title")

df

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_id,api_id,artists,duration_ms,isrc,ean,upc,href,available_countries,popularity
track_title,,,,,,,,,,,,,,,,,,,,
Nothing is Safe,ed8c97de-d12d-41c1-bd96-cb3c924573ef,0.082300,0.920,0.489,0.05200,0.0626,-9.190,0.0731,125.973,0.3270,ed8c97de-d12d-41c1-bd96-cb3c924573ef,ed8c97de-d12d-41c1-bd96-cb3c924573ef,[{'id': 'c73f1acd-5904-40d6-91ed-4776f9976d52'...,297249,USSUB1933002,NaN,NaN,https://open.spotify.com/track/52UthFOfrbtnjBM...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",44
The Way I Am,b226107f-7649-4765-b982-41bc4817859a,0.144000,0.785,0.871,0.00000,0.3090,-2.692,0.2650,87.248,0.3150,b226107f-7649-4765-b982-41bc4817859a,b226107f-7649-4765-b982-41bc4817859a,[{'id': '36e741c0-7572-4ec4-84da-d8ed2a001589'...,290426,USIR10000589,NaN,NaN,https://open.spotify.com/track/23wfXwnsPZYe5A1...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",71
Doesn't Make It Alright - 2015 Remaster,9508aac8-8cd7-48b2-9036-76981b6fd4e6,0.022000,0.713,0.380,0.13000,0.3080,-10.599,0.0668,139.728,0.8330,9508aac8-8cd7-48b2-9036-76981b6fd4e6,9508aac8-8cd7-48b2-9036-76981b6fd4e6,[{'id': '7d588c1e-284e-41e2-9878-ce107592fbb1'...,206080,GBAYK1400006,NaN,NaN,https://open.spotify.com/track/54CPnVfOQiDo4mn...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",35
Refugee,0dd7fb45-8d19-4ff9-81a5-1363363dea7c,0.398000,0.394,0.664,0.00439,0.2010,-8.213,0.0357,129.822,0.2420,0dd7fb45-8d19-4ff9-81a5-1363363dea7c,0dd7fb45-8d19-4ff9-81a5-1363363dea7c,[{'id': '6988e20e-ca58-4025-b68e-d0183068978f'...,217693,GBDYB0300111,NaN,NaN,https://open.spotify.com/track/15qTTqTKYsqJRhz...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",37
Drug Ballad,51283d70-0b24-4cee-b2c9-a06aa26dab99,0.023900,0.913,0.700,0.00000,0.1250,-3.411,0.0589,104.759,0.6400,51283d70-0b24-4cee-b2c9-a06aa26dab99,51283d70-0b24-4cee-b2c9-a06aa26dab99,[{'id': '36e741c0-7572-4ec4-84da-d8ed2a001589'...,300266,USIR10000587,NaN,NaN,https://open.spotify.com/track/7tEoJKbYdIHBfn7...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"The Island, Pt. I (Dawn)",84ff94d3-b044-48fe-b6b0-7635f380542b,0.000062,0.578,0.893,0.00629,0.1470,-3.615,0.0557,126.018,0.3730,84ff94d3-b044-48fe-b6b0-7635f380542b,84ff94d3-b044-48fe-b6b0-7635f380542b,[{'id': '8bd7e37d-5aa3-444d-9719-0ca0e8d71fb3'...,320173,GBAHT1000129,NaN,NaN,https://open.spotify.com/track/1Thv8uCYzyOFC7P...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",52
Comprachicos,18900c29-0a3f-4b85-816a-371cb105f32e,0.000024,0.450,0.955,0.00374,0.3670,-3.620,0.0593,174.017,0.5880,18900c29-0a3f-4b85-816a-371cb105f32e,18900c29-0a3f-4b85-816a-371cb105f32e,[{'id': '8bd7e37d-5aa3-444d-9719-0ca0e8d71fb3'...,168546,GBAHT1000131,NaN,NaN,https://open.spotify.com/track/2ZIJUwprFZrAaZC...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",34
Self vs Self (feat. In Flames),d720025a-3026-45a8-90ca-598485d5be47,0.000005,0.438,0.989,0.00198,0.9410,-2.354,0.0959,174.012,0.3320,d720025a-3026-45a8-90ca-598485d5be47,d720025a-3026-45a8-90ca-598485d5be47,[{'id': '8bd7e37d-5aa3-444d-9719-0ca0e8d71fb3'...,285706,GBAHT1000134,NaN,NaN,https://open.spotify.com/track/2lN6G35gsXkA3xz...,"AR,AU,AT,BE,BO,BR,BG,CA,CL,CO,CR,CY,CZ,DK,DO,D...",54


In [68]:
# Rescale columns to 0-1 range
from sklearn.preprocessing import MinMaxScaler

scaled_df = df.drop(
    columns=[
        "id",
        "api_id",
        "track_id",
        "ean",
        "upc",
        "available_countries",
        "href",
        "isrc",
        "artists",
    ]
)
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(
    scaler.fit_transform(scaled_df),
    index=scaled_df.index,
    columns=scaled_df.columns,
)

In [69]:
hvplot.scatter_matrix(scaled_df)

:GridMatrix   [X,Y]
   :Histogram   [valence]   (Count)

In [70]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(scaled_df)
embedding_df = pd.DataFrame(embedding, index=scaled_df.index, columns=["x", "y"])

d:\Programming\spotify-analysis\.pixi\envs\default\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [71]:
scaled_df["energy"]

track_title
Nothing is Safe                            0.489560
The Way I Am                               0.874152
Doesn't Make It Alright - 2015 Remaster    0.379820
Refugee                                    0.665747
Drug Ballad                                0.701991
                                             ...   
The Island, Pt. I (Dawn)                   0.896301
Comprachicos                               0.958722
Self vs Self (feat. In Flames)             0.992952
The Fountain (feat. Steven Wilson)         0.960735
Encoder                                    0.832874
Name: energy, Length: 2751, dtype: float64

In [98]:
def extract_first_name_ast(x):
    try:
        if pd.isna(x) or x == "":
            return None
        # Use ast.literal_eval to safely parse string representation
        artists = ast.literal_eval(x)
        if artists and len(artists) > 0:
            return artists[0]["name"]
        return None
    except (ValueError, SyntaxError, KeyError, TypeError, IndexError):
        return None


first_names = df.artists.apply(extract_first_name_ast)

first_names

track_title
Nothing is Safe                               clipping.
The Way I Am                                     Eminem
Doesn't Make It Alright - 2015 Remaster    The Specials
Refugee                                       Oi Va Voi
Drug Ballad                                      Eminem
                                               ...     
The Island, Pt. I (Dawn)                       Pendulum
Comprachicos                                   Pendulum
Self vs Self (feat. In Flames)                 Pendulum
The Fountain (feat. Steven Wilson)             Pendulum
Encoder                                        Pendulum
Name: artists, Length: 2751, dtype: object

In [100]:
test = scaled_df.copy()

In [104]:
test["artist"] = first_names.astype("category")
test

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,popularity,artist
track_title,,,,,,,,,,,,
Nothing is Safe,0.082626,0.954357,0.489560,0.052953,0.045719,0.786833,0.080241,0.606593,0.332655,0.198650,0.483516,clipping.
The Way I Am,0.144574,0.814315,0.874152,0.000000,0.301746,0.939889,0.290889,0.420122,0.320448,0.194090,0.780220,Eminem
Doesn't Make It Alright - 2015 Remaster,0.022084,0.739627,0.379820,0.132383,0.300707,0.753645,0.073326,0.672827,0.847406,0.137722,0.384615,The Specials
Refugee,0.399595,0.408714,0.665747,0.004470,0.189526,0.809846,0.039188,0.625127,0.246185,0.145483,0.406593,Oi Va Voi
Drug Ballad,0.023991,0.947095,0.701991,0.000000,0.110557,0.922954,0.064654,0.504442,0.651068,0.200666,0.615385,Eminem
...,...,...,...,...,...,...,...,...,...,...,...,...
"The Island, Pt. I (Dawn)",0.000057,0.599585,0.896301,0.006405,0.133416,0.918149,0.061142,0.606810,0.379451,0.213970,0.571429,Pendulum
Comprachicos,0.000019,0.466805,0.958722,0.003809,0.362012,0.918031,0.065093,0.837938,0.598169,0.112638,0.373626,Pendulum
Self vs Self (feat. In Flames),0.000000,0.454357,0.992952,0.002016,0.958437,0.947851,0.105269,0.837913,0.337742,0.190936,0.593407,Pendulum


In [ ]:
color_options = test.columns.tolist()

# Create dropdown widget
color_dropdown = pn.widgets.Select(
    name="Color Variable", value="loudness", options=color_options
)


# Function to create plot based on dropdown selection
def create_plot(color_var):
    if color_var == "artist":
        colormap = "Category20"
    else:
        colormap = "viridis"
    return embedding_df.hvplot.scatter(
        x="x",
        y="y",
        hover_cols=["track_title", "artist"],
        title=f"UMAP Embedding of Tracks (colored by {color_var})",
        color=test[color_var],
        width=800,
        height=600,
        cmap=colormap,
    )


# Bind the plot function to the dropdown
interactive_plot = pn.bind(create_plot, color_dropdown)

# Create the dashboard
dashboard = pn.Column("## Interactive UMAP Plot", color_dropdown, interactive_plot)

# Display the dashboard
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'7c906fd9-9180-4544-8317-ebc3a67d7473': {'version…